In [ ]:
from datasets import concatenate_datasets, load_from_disk, DatasetDict

In [ ]:
article_classification_task_dataset = load_from_disk('../datasets/tasks/article_classification_task_dataset/').shuffle(seed=42)
institution_classification_task_dataset = load_from_disk('../datasets/tasks/institution_classification_task_dataset/').shuffle(seed=42)
keyword_generation_task_dataset = load_from_disk('../datasets/tasks/keyword_generation_task_dataset/').shuffle(seed=42)
people_classification_task_dataset = load_from_disk('../datasets/tasks/people_classification_task_dataset/').shuffle(seed=42)

In [ ]:
article_classification_task_dataset

In [ ]:
def get_pandas(dataset):
    return {split: dataset[split].to_pandas() for split in dataset.keys()}

In [ ]:
pandas_article_classification_task_dataset = get_pandas(article_classification_task_dataset)
pandas_institution_classification_task_dataset = get_pandas(institution_classification_task_dataset)
pandas_keyword_generation_task_dataset = get_pandas(keyword_generation_task_dataset)
pandas_people_classification_task_dataset = get_pandas(people_classification_task_dataset)

In [ ]:
for split in article_classification_task_dataset.keys():
    print(split, pandas_article_classification_task_dataset[split].groupby(['label']).size(), sep='\n', end='\n\n')

In [ ]:
institution_classification_task_dataset

In [ ]:
keyword_generation_task_dataset

In [ ]:
people_classification_task_dataset

In [ ]:
combined_tasks = DatasetDict()

In [ ]:
for split, size in zip(['train', 'validation', 'test'], [16000, 3000, 1500]):
    combined_tasks[split] = article_classification_task_dataset[split].filter(lambda article: article['label'] == 1).select(range(size))
    combined_tasks[split] = concatenate_datasets([combined_tasks[split], article_classification_task_dataset[split].filter(lambda article: article['label'] == 0).select(range(size//2))])
combined_tasks = combined_tasks.remove_columns(['label', 'type'])
#for split, size in zip(['train', 'validation', 'test'], [10000, 1400, 740]):
#    combined_tasks[split] = concatenate_datasets([combined_tasks[split], people_classification_task_dataset[split].select(range(size)).remove_columns(['type'])])
#for split, size in zip(['train', 'validation', 'test'], [7000, 900, 490]):
#    combined_tasks[split] = concatenate_datasets([combined_tasks[split], institution_classification_task_dataset[split].select(range(size)).remove_columns(['type'])])
#for split, size in zip(['train', 'validation', 'test'], [10000, 3000, 1500]):
#    combined_tasks[split] = concatenate_datasets([combined_tasks[split], keyword_generation_task_dataset[split].select(range(size)).remove_columns(['type'])])

In [ ]:
combined_tasks.shuffle(seed=42).save_to_disk('../datasets/tasks/combined_task_dataset')

In [ ]:
print(combined_tasks.shuffle(seed=42)['train'][0]['text'])

In [ ]:
combined_tasks = load_from_disk('../datasets/tasks/combined_task_dataset/')

In [ ]:
combined_tasks

In [ ]:
combined_tasks_io = combined_tasks.map(lambda task: {
    'input': task['text'][:task['text'].rfind(':')+1],
    'output': task['text'][task['text'].rfind(':')+1:],
}, remove_columns='text')

In [ ]:
print(combined_tasks_io['train'][1]['input'])

In [ ]:
combined_tasks_io.filter(lambda task: 'korrupciós címkék:' not in task['input'])

In [ ]:
#combined_tasks_io.filter(lambda task: 'korrupciós címkék:' not in task['input']).save_to_disk('datasets/tasks/combined_task_dataset_io_notags')

In [ ]:
combined_tasks_io.filter(lambda task: '[korrupció klasszifikáció]' in task['input']).save_to_disk('../datasets/tasks/combined_task_dataset_io_notags')

In [ ]:
combined_tasks_io.filter(lambda task: 'korrupciós címkék:' not in task['input'])